In [1]:
from pyzotero import zotero
from zotero2qmd.zotero2qmd import item2main, main2qmd, filter_pubs
from collections import Counter
import re
import yaml
from pathlib import Path

In [2]:
keypath = Path(".zotero")
with keypath.open() as keyfile:
    key_text = keyfile.readline().strip()

In [3]:
zot = zotero.Zotero(library_id='7642731', library_type='user', api_key=key_text)

In [4]:
pubs = zot.publications()

In [5]:
filtered_pubs = filter_pubs(pubs)

In [6]:
all_mains = [item2main(x) for x in filtered_pubs]

In [7]:
all_mains[0]

{'params': {'key': '6IQH5VV5', 'notes': 'DOI: 10.5281/zenodo.10393038'},
 'author': [{'name': {'given': 'Josef', 'family': 'Fruehwald'}}],
 'title': 'syllabifyr: v0.1.1',
 'date': '2020-10-24',
 'date-format': 'YYYY',
 'description': 'v0.1.1',
 'abstract': 'The goal of `syllabifyr` is to provide tidy syllabification of phonetic transcriptions. So far, only CMU dict transcriptions are supported.',
 'citation': {'type': 'software',
  'issued': '2020-10-24',
  'url': 'https://jofrhwld.github.io/syllabifyr/',
  'version': 'v0.1.1'}}

In [8]:
def make_file_names(all_mains):
    first_aut = [x["author"][0]["name"]["family"] for x in all_mains]
    years = [x["date"].split("-")[0] for x in all_mains]
    keys = [x["params"]["key"] for x in all_mains]

    all_stem = [f"{aut}_{year}_{key}" for aut, year, key in zip(first_aut, years, keys)]

    all_stem = [re.sub(r"_$", "", x) for x in all_stem]
    return all_stem

In [9]:
all_stems = make_file_names(all_mains)

In [10]:
all_stems

['Fruehwald_2020_6IQH5VV5',
 'Fruehwald_2023_MW4ESNC8',
 'Rosenfelder_2011_M3E7A7LS',
 'Rosenfelder_2015_K57PJ2RE',
 'Rosenfelder_2022_TRXWWXD2',
 'Fruehwald_2023_GY5I9QMZ',
 'Fruehwald_2019_URYFVWHE',
 'Tanner_2020_GH2LZM4Y',
 'Purse_2022_LIGPGR5R',
 'Mielke_2019_U5C56KNR',
 'Labov_2013_QZP9CAP8',
 'Fruehwald_2013_W28LR5KT',
 'Fruehwald_2011_LPQN5R3M',
 'Fruehwald_2022_H347UNLS',
 'Fruehwald_2017_ETG527QQ',
 'Fruehwald_2012_7HNEQZ7H',
 'Fruehwald_2008_IZLBZKZV',
 'Fruehwald_2007_MG8DFJEK',
 'Boyd_2021_733Q4MMI',
 'Fruehwald_2016_TCTQU8FS',
 'Wieling_2016_FU9RXCUV',
 'Sneller_2019_8ERVHBSG',
 'Fruehwald_2015_TJI9T9W6',
 'Fruehwald_2017_258NF56V',
 'Fruehwald_2016_34D4IRNY',
 'Fruehwald_2013_H3BZPAT7']

In [11]:
out_path = Path("research","papers")

In [12]:
for stem, item in zip(all_stems, all_mains):
    out_file = out_path.joinpath(stem).with_suffix(".qmd")
    if not out_file.exists():
        with out_file.open(mode = "w"):
            qmd_string = "---\n"+yaml.dump(item)+"\n---"
            out_file.write_text(qmd_string)